# ICU / CLDR / intl

## Úvod

Jistě již Vás někdy potkala některá z následujících situací:

* Našli jste Česko v seznamu zemí zahrabané až kdesi za Zimbabwe.
* Zvolili jste špatné datum, protože týden v grafickém kalendáři nezačínal pondělím.
* Nebyli jste si jisti, která část data je den a která měsíc.
* Zkopírovali jste „2 souborů“.
* Přišel Vám e-mail nadepsaný „Byl(a) jste s našimi službami spokojen(a)?“
* Nevěděli jste, v jakém časovém pásmu jsou uvedeny časy v jízdním řádu.
* Místo popisků tlačítek jste viděli jen výpustku.
* Nechtěně jste zadali stovky místo jednotek kvůli desetinné tečce místo čárky.

Nejen to nám pomáhá vyřešit knihovna ICU a databáze CLDR.

Spousta programátorů by byla ráda, kdyby všichni na světě používali jednotný systém prakticky všeho, nejlépe i jen jediný jazyk. Některým by snad stačilo, kdyby všichni aspoň používali latinku a arabské číslice. Jenže… lidi nezměníte a software děláte pro ně.

Podpora cizích jazyků je zásadní. Nesmí se stát, že rozdíly v jazyce, znakové sadě či národním prostředí způsobí, že aplikace nebude fungovat. Dobře udělaná lokalizace a přizpůsobení národnímu prostředí, které není zásadní pro funkčnost, je dalším krokem: výrazně zlepšuje to, jak je nám používání aplikace příjemné.

### Příklady

* Nejhorší: Kvůli rozdílu v nastavení oddělovačů tisíců nepůjde odeslat formulář s částkou.
* Nepříjemné: Nadpisy článků, které nejsou latinkou, vytvoří nesrozumitelné adresy: _------_ místo _privet_.
* Kosmetické: Datum se zobrazí s jiným pořadím jednotlivých součástí.

### Co už jsem slyšel

* Sestavujte věty tak, aby nebylo potřeba měnit tvary slov.
* Nemůžeme ty ruské e-maily posílat napsané latinkou?

## IntlDateFormatter

In [59]:
$dateFormat = IntlDateFormatter::LONG;
$timeFormat = IntlDateFormatter::LONG;
$timeZone = new DateTimeZone('Europe/Prague');
$formatter = new IntlDateFormatter('cs_CZ', $dateFormat, $timeFormat, $timeZone);
$dateTime = new DateTime;
$formatter->format($dateTime);

"19. října 2018 1:49:42 SELČ"

## NumberFormatter

In [40]:
$out = [];
$decimalFormatter = new NumberFormatter('cs_CZ', NumberFormatter::DECIMAL);
$out[] = $decimalFormatter->format(123456789.0123456);
$decimalFormatter->setAttribute(NumberFormatter::MAX_FRACTION_DIGITS, 4);
$out[] = $decimalFormatter->format(123456789.0123456);
$out;

[
  "123 456 789,012",
  "123 456 789,0123",
]

In [37]:
$spelloutFormatter = new NumberFormatter('cs_CZ', NumberFormatter::SPELLOUT);
$spelloutFormatter->format(123456);

"sto dvacet tři tisíc čtyři sta padesát šest"

In [39]:
$spelloutFormatter = new NumberFormatter('cs_CZ', NumberFormatter::CURRENCY);
$spelloutFormatter->format(1234.5678);

"1 234,57 Kč"

## Collator

In [43]:
$birds = ['chřástal', 'čížek', 'ťuhýk', 'datel', 'čáp', 'turpan', 'cetie'];
sort($birds);
$birds;

[
  "cetie",
  "chřástal",
  "datel",
  "turpan",
  "čáp",
  "čížek",
  "ťuhýk",
]

In [44]:
$birds = ['chřástal', 'čížek', 'ťuhýk', 'datel', 'čáp', 'turpan', 'cetie'];
$collator = new Collator('cs_CZ');
$collator->sort($birds);

$birds;

[
  "cetie",
  "čáp",
  "čížek",
  "datel",
  "chřástal",
  "ťuhýk",
  "turpan",
]

Příklad ze života: Telefonní seznam některých starších Nokií správně řadil _ch_ až za _h_. Vyhledávání však zpracovával postupně po jednotlivých písmenech. Jména obsahující _ch_ tak nebylo možné vyhledat zadáním _ch_ do vyhledávacího pole.

## MessageFormatter

Už jsem slyšel i výzvy: tak ty věty pište tak, aby se slova neměnila v závislosti na čísle.

> Počet jablek, která měla babka: 4.

Jako vážně?

In [1]:
$appleCount = 5;
"Měla babka $appleCount jablek.";

"Měla babka 5 jablek."

In [3]:
$appleCount = 1.1;
"Měla babka $appleCount jablek.";

"Měla babka 1.1 jablek."

In [46]:
$countPattern = 'Měla babka {apple_count, number} {apple_count, plural, one {jablko} '  .
                                                                            'few {jablka} ' . 
                                                                            'many {jablka} ' .
                                                                            'other {jablek}}.';
$formatter = new MessageFormatter('cs_CZ', $countPattern);

$appleCounts = [1, 4, 5, 5.5, 0];
$messages = [];
foreach($appleCounts as $appleCount) {
    $messages[] = $formatter->format(['apple_count' => $appleCount]);
}

$messages;

[
  "Měla babka 1 jablko.",
  "Měla babka 4 jablka.",
  "Měla babka 5 jablek.",
  "Měla babka 5,5 jablka.",
  "Měla babka 0 jablek.",
]

In [47]:
$genderPattern = '{gender, select, m {{subject} měl jablka.} ' .
                                  'f {{subject} měla jablka.} ' .
                                  'n {{subject} mělo jablka.} ' .
                                  'other {{subject} si nezaslouží jablka.}}';
$formatter = new MessageFormatter('cs_CZ', $genderPattern);

$examples = [['subject' => 'Dědek', 'gender' => 'm'],
             ['subject' => 'Babka', 'gender' => 'f'],
             ['subject' => 'Vnouče', 'gender' => 'n']];
$messages = [];
foreach($examples as $example) {
    $messages[] = $formatter->format($example);
}
$messages;

[
  "Dědek měl jablka.",
  "Babka měla jablka.",
  "Vnouče mělo jablka.",
]

## Normalizer

In [74]:
$out = [];
$original = 'k' . 'u' . '̊' . 'n' . '̌';
$out[] = $original;
$out[] = mb_strlen($original);
$out[] = Normalizer::isNormalized($original);

$normalized = Normalizer::normalize($original);
$out[] = $normalized;
$out[] = mb_strlen($normalized);
$out[] = Normalizer::isNormalized($normalized);

$out[] = $original == $normalized;
$out;

[
  "kůň",
  5,
  false,
  "kůň",
  3,
  true,
  false,
]

Narazil jsem na chybu v jádře Ruby: při vytvoření souboru na souborovém systému HFS souborový systém název souboru normalizuje, když se předá nenormalizovaný. Ruby si ale ponechalo původní nenormalizovaný název, pod kterým zapsaný soubor nebylo možné nalézt.

## Transliterate

## IntlChar

In [81]:
$out = [];
$chars = ['ň', '℃', 'ふ', '갨', '🍺'];
foreach ($chars as $char) {
    $out[] = IntlChar::charname($char);
}
$out;

[
  "LATIN SMALL LETTER N WITH CARON",
  "DEGREE CELSIUS",
  "HIRAGANA LETTER HU",
  "HANGUL SYLLABLE GAELS",
  "BEER MUG",
]

In [84]:
$out = [];

$chars = ['Ň', 'ň', '🍺'];
foreach ($chars as $char) {
    $out[] = [IntlChar::islower($char), IntlChar::isgraph($char)];
}

$out;

[
  [
    false,
    true,
  ],
  [
    true,
    true,
  ],
  [
    false,
    true,
  ],
]

In [93]:
$out = [];
$char = ' ';
$trimmed = trim($char);
$out[] = mb_strlen($trimmed);
$out[] = IntlChar::isWhiteSpace($char);
$out[] = IntlChar::isUWhiteSpace($char);
$out;

[
  1,
  false,
  true,
]

## CIDR? Počty slov? …